# Troubled Times

I am not to a disaster, but mearely the challenges of accurately "measuring" small time differences with microcontrollers.

Depending on clock rate, microcontrollers can resolve sub-microsecond intervals. Equally, using counters, they can track years or even millenia, provided that the power does not go out and someone is there to look at the result.

## Problem

Representing such intervals is also quite easy. A 64-bit integer can represent 584,942 years with microsecond resolution. The problem is that most microcontrollers use 32-bit integers. MicroPython "looses" a bit or two to represent the datatype. An additional bit is used for the sign, hence we are left with 29 bits. That's sufficient to represent just 9 minutes with microsecond resolution. With millisecond resolution we get about 9000 minutes or a little over 6 days.

Measuring time is typically implemented by arbitrarily resetting a clock variable to zero (usually on powerup) and incrementing its value every millisecond or so. MicroPython has a function `time.ticks_ms()` that returns the time in microseconds since the device was turned on.

For example, to measure the duration of button presses (or some sensor input, let's say the duration of a flash of light), we take the time when the button was pressed, and again when it was released and compute the difference.

```python
from time import ticks_ms

# wait for button press
# ...
start = ticks_ms()

# wait for button released
# ...
stop = ticks_ms()

# calculate the duration of the button press
duration = stop - start
```

This works. Except that every six days we get incorrect results since the timer "overflows". Needless to say that such bugs are extremely difficult to diagnose since they occur so infrequently. Because of this they often escape testing, showing up only after the product has been relased to customers with possibly disastrous results.

# `time.monotonic`

C-Python has a function `time.monotonic()` that returns time (with arbitrary offset) in seconds as a 64-bit floating point number. With floating point the resolution decreases for long durations, but to less than microseconds after half a million years, but most of us won't be around to notice.

[CircuitPython](https://circuitpython.org/), a fork of MicroPython, also offers `time.monotonic()` but returns a 32-bit float rather than 64-bits. Now the resolution drops quite quickly, well within our lifetime:

In [1]:
%connect huzzah32

# simulate `time.monotonic()` just after powerup:
start = 0

# simulate 1ms interval:
stop = start+0.001

print("duration =", stop-start)

Connected to huzzah32 @ serial:///dev/ttyUSB0
duration = 0.001


The measured duration is 1ms, as it should be.

Now let's repeat the same experiment but after the micrcontroller has been running for a day:

In [1]:
# simulate `time.monotonic()` just after a day:
start = 3600*24

# simulate 1ms interval:
stop = start+0.01

print("duration =", stop-start)

duration = 0.0


Zero? That's clearly wrong and the consequence of the limited resolution of MicroPython 32-bit floats. If you play around with the above code you find that after one day, the smallest interval that can be resolved is 4ms, albeit the duration is reported as 7ms.

Perhaps you do not need millisecond resolution. But even 10ms cannot be resolved after 4 days.

# Solution

There are several solutions for this. We could allocate more bits for representing time, for example using Python bigints. In fact, CircuitPython's `time.monotonic_ns()` does just this.

It works fine and has the great benefit of being standard Python (i.e. code works also in C-Python) but has potential drawbacks. First, bigints are less efficient than (small) ints or floats. Secondly, and more importantly, they are allocated on the heap. Because of this they cannot be used in [interrupt service routines](https://docs.micropython.org/en/latest/reference/isr_rules.html#isr-rules). Since interrupts are precisely a situation where measuring time is often needed, that's a quite significant gotcha. 

Note that CircuitPython does not implement interrupts in user code. That's also a solution, if you don't need them.

# Ticks

MicroPython chooses a different route. `time.ticks_ms()` and `time.ticks_us()` both return time as small ints. Because of this they "roll-over" after a few minutes or days. The special function `time.ticks_diff()` takes into account the roll over.

**Example:**

```python
start = time.ticks_us()  # or ticks_ms

# do whatever you want to measure the duration of
# ...

stop = time.ticks_us()
```

To get the duration

```python
duration = time.ticks_diff(stop, start)
```

Rather than the (incorrect)

```python
duration = stop - start  # WRONG!
```

This works correctly as long as no more than one roll-over occurs during the measurement interval.

Admittedly a bit contorted, but it works correctly, is efficient, and can be used even in interrupt service routines.
